In [ ]:
from langchain_core.tools import InjectedToolArg
from typing import Annotated
from langchain_groq import ChatGroq
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
from langchain_google_genai import ChatGoogleGenerativeAI
import requests
import dotenv
import os

dotenv.load_dotenv()


os.environ["GOOGLE_API_KEY"] = "xyz"

# Initialize the Gemini model
# Note: Popular models include 'gemini-2.0-flash' or 'gemini-1.5-pro'
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0.7
)


Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.


In [4]:
@tool
def get_conversion_factor(base_currency: str, target_currency: str) -> float:
    """
    This function fetches the currency conversion factor between a given base currency and a target currency
    """
    url = f"https://v6.exchangerate-api.com/v6/xyz/pair/{base_currency}/{target_currency}"

    response = requests.get(url)
    return response.json()
    


In [57]:
@tool
def convert(base_currency_value: int, conversion_rate: Annotated[float, InjectedToolArg]) -> float: # here InjectedToolArg used because if we use this funciton or tool with out InjectedToolArg then both tools will run togather and because it runs both the tools togather , so the output of first tool which in coversation_rate will not be a paert of input of 2nd convert tool so for that field in 2nd tool , llm will give any random value so answer will not be correct 
    # thats why we use InjectedToolArg, what it does , it says llm do not try to fill this"conversation_rate" field , I (developer) will inject this value after running first tool.
  """
  given a currency conversion rate this function calculates the target currency value from a given base currency value
  """
  return base_currency_value * conversion_rate

In [46]:
convert.args # see here when i ask for arguments it will only show the first argument and not the argument with InjectedToolArg, same will be done with llm

{'base_currency_value': {'title': 'Base Currency Value', 'type': 'integer'},
 'conversion_rate': {'anyOf': [{'type': 'number'}, {'type': 'null'}],
  'title': 'Conversion Rate'}}

In [47]:
conversion_factor = get_conversion_factor.invoke({'base_currency':'USD','target_currency':'INR'})
conversion_factor

{'result': 'success',
 'documentation': 'https://www.exchangerate-api.com/docs',
 'terms_of_use': 'https://www.exchangerate-api.com/terms',
 'time_last_update_unix': 1768867201,
 'time_last_update_utc': 'Tue, 20 Jan 2026 00:00:01 +0000',
 'time_next_update_unix': 1768953601,
 'time_next_update_utc': 'Wed, 21 Jan 2026 00:00:01 +0000',
 'base_code': 'USD',
 'target_code': 'INR',
 'conversion_rate': 90.9589}

In [48]:
conversion_factor['conversion_rate']

90.9589

In [49]:
convert.invoke({'base_currency_value':10, 'conversion_rate':conversion_factor["conversion_rate"]})

909.5889999999999

In [50]:
# TOOL BINDING

In [51]:
llm_with_tools = llm.bind_tools([get_conversion_factor, convert])
llm_with_tools

RunnableBinding(bound=ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 32768, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': True, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x00000241BBDD55B0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000241BBDD5EB0>, model_name='openai/gpt-oss-20b', model_kwargs={}, groq_api_key=SecretStr('**********')), kwargs={'tools': [{'type': 'function', 'function': {'name': 'get_conversion_factor', 'description': 'This function fetches the currency conversion factor between a given base currency and a target currency', 'parameters': {'properties': {'base_currency': {'type': 'string'}, 'target_currency': {'type': 'string'}}, 'required': ['base_currency', 'target_currency'], 'type': 'object'}}}, {'type': 'function', 'function': {'name': 'convert', 'descript

In [52]:
messages = [
    HumanMessage(content="What is the conversion factor between INR and USD, and based on that convert 10 INR to USD")
]
messages

[HumanMessage(content='What is the conversion factor between INR and USD, and based on that convert 10 INR to USD', additional_kwargs={}, response_metadata={})]

In [53]:
ai_message = llm_with_tools.invoke(messages)


In [54]:
ai_message.tool_calls

[{'name': 'get_conversion_factor',
  'args': {'base_currency': 'INR', 'target_currency': 'USD'},
  'id': 'fc_749931fc-53e2-4890-98be-3664a184b57f',
  'type': 'tool_call'}]

In [ ]:
import json

for tool_call in ai_message.tool_calls:
  # execute the 1st tool and get the value of conversion rate
  if tool_call['name'] == 'get_conversion_factor':
    tool_message1 = get_conversion_factor.invoke(tool_call)
    # fetch this conversion rate
    conversion_rate = json.loads(tool_message1.content)['conversion_rate']
    # append this tool message to messages list
    messages.append(tool_message1)
  # execute the 2nd tool using the conversion rate from tool 1
  if tool_call['name'] == 'convert':
    # fetch the current arg
    tool_call['args']['conversion_rate'] = conversion_rate
    tool_message2 = convert.invoke(tool_call)
    messages.append(tool_message2)



In [ ]:
llm_with_tools.invoke(messages).content

In [ ]:
from langchain.agents import initialize_agent, AgentType

# Step 5: Initialize the Agent ---
agent_executor = initialize_agent(
    tools=[get_conversion_factor, convert],
    llm=llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,  # using ReAct pattern
    verbose=True  # shows internal thinking
)

In [ ]:
# --- Step 6: Run the Agent ---
user_query = "Hi how are you?"

response = agent_executor.invoke({"input": user_query})